Import libraries

In [1]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras import layers, models, optimizers
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

Load the dataset

In [2]:
# Folder path 
folder_path = 'D:\FPTUni\SP24\ADY201m\Lab05\ADY201m_Lab05_SE183256'
# Import the dataset
train_data = pd.read_csv(os.path.join(folder_path, 'train.csv'))
test_data = pd.read_csv(os.path.join(folder_path, 'test.csv'))
output_path = os.path.join(folder_path, 'submission.csv')

In [3]:
# Save ID of test data
test_id = test_data['Id']

In [4]:
# Drop unnecessary columns
train_data = train_data.drop('Id', axis=1)

House Price Distribution

In [5]:
# Setup threshold for missing values

# === IMPORTANT ===


# Here we will put the variable to choose the threshold
threshold = 0.5


# === IMPORTANT ===

Numerical data distribution

In [6]:
# Because MSSubClass is int64 but it is a categorical variable, so we need to convert it to string
train_data['MSSubClass'] = train_data['MSSubClass'].astype(str)

In [7]:
# Get the categorical columns
categorical_cols = train_data.select_dtypes(include=['object']).columns.tolist()
# Delete all categorical columns
train_data = train_data.drop(categorical_cols, axis=1)

In [8]:
# Compute the correlation matrix with SalePrice
corr_matrix = train_data.corr()
corr_matrix['SalePrice'].sort_values(ascending=False)
# ABS of correlation coefficient
# Get the features that have the correlation coefficient with SalePrice greater than the threshold
selected_features = corr_matrix['SalePrice'][corr_matrix['SalePrice'].abs() > threshold].index.tolist()
# Display the selected features

In [9]:
# Create the scaler
scaler = StandardScaler()
scaler2 = StandardScaler()
# Fit the scaler to the train data
scaler.fit(train_data[selected_features])
# Transform the train data
train_data_scaled = scaler.transform(train_data[selected_features])

In [10]:
# Convert test data to same format with train data to predict, ( test data doest not have SalePrice column)
# remove saleprice column from selected_features and make new selected_features2
selected_features2 = selected_features.copy()
selected_features2.remove('SalePrice')
# if Id have in test data, we need to remove it, if not, we don't need to remove it
if 'Id' in test_data.columns:
    test_data = test_data.drop('Id', axis=1)
scaler2.fit(test_data[selected_features2])
test_data_scaled = scaler2.transform(test_data[selected_features2])

In [34]:
# Prepare the data
y_train = train_data_scaled[:, -1]
x_train = train_data_scaled[:, :-1]

# Split the train data into train and test data
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [44]:
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(x_train, y_train)
predictions_rf = model_rf.predict(x_test)
mse_rf = mean_squared_error(y_test, predictions_rf)
print("Root Mean Squared Error (Random Forest):", np.sqrt(mse_rf))

Epoch 1/200
42/42 [==============================] - 1s 4ms/step - loss: 0.3772 - mae: 0.3911 - val_loss: 0.2955 - val_mae: 0.2901
Epoch 2/200
42/42 [==============================] - 0s 2ms/step - loss: 0.2217 - mae: 0.2830 - val_loss: 0.2628 - val_mae: 0.2539
Epoch 3/200
42/42 [==============================] - 0s 2ms/step - loss: 0.1903 - mae: 0.2652 - val_loss: 0.2652 - val_mae: 0.2456
Epoch 4/200
42/42 [==============================] - 0s 2ms/step - loss: 0.1760 - mae: 0.2614 - val_loss: 0.2367 - val_mae: 0.2414
Epoch 5/200
42/42 [==============================] - 0s 2ms/step - loss: 0.1571 - mae: 0.2500 - val_loss: 0.2386 - val_mae: 0.2469
Epoch 6/200
42/42 [==============================] - 0s 2ms/step - loss: 0.1536 - mae: 0.2501 - val_loss: 0.2387 - val_mae: 0.2416
Epoch 7/200
42/42 [==============================] - 0s 1ms/step - loss: 0.1433 - mae: 0.2463 - val_loss: 0.1571 - val_mae: 0.2315
Epoch 8/200
42/42 [==============================] - 0s 2ms/step - loss: 0.1533 - m

In [38]:
# Save model to file
model01.save('model01.house_price_prediction')

INFO:tensorflow:Assets written to: model01.house_price_prediction\assets


INFO:tensorflow:Assets written to: model01.house_price_prediction\assets


In [45]:
# Predict using the trained model
result_scaled = model01.predict(test_data_scaled)

# Combine scaled test data with scaled result
combined_data_scaled = np.concatenate((test_data_scaled, result_scaled), axis=1)
# Unscale the combined data to get the final result
result = scaler.inverse_transform(combined_data_scaled)[:, -1]

# Kiểm tra result có cái nào trống hoặc là số âm không, nếu có thì thay nó bằng 0
result = np.where(result < 0, 0, result)
result = np.where(np.isnan(result), 0, result)

46/46 [==============================] - 0s 645us/step


In [46]:
# Create the submission file
submission = pd.DataFrame({'Id': test_id, 'SalePrice': result})
# Save the submission file
submission.to_csv(output_path, index=False)

In [47]:
# Predict using the trained model
result_scaled = model01.predict(x_test)
# Unscale the combined data to get the final result
result = scaler.inverse_transform(np.concatenate((x_test, result_scaled), axis=1))[:, -1]
y_test_unscaled = scaler.inverse_transform(np.concatenate((x_test, y_test.reshape(-1, 1)), axis=1))[:, -1]
# Display random 100 results
for i in range(10):
    print(f'Predicted: {result[i]}, Actual: {y_test_unscaled[i]}')

# Evaluate the model performance using Mean Squared Error
mse = mean_squared_error(y_test_unscaled, result)

# Print the Root Mean Squared Error
print("Root Mean Squared Error:", np.sqrt(mse))

5/5 [==============================] - 0s 1ms/step
Predicted: 147307.8910161949, Actual: 154500.0
Predicted: 323625.4334910995, Actual: 325000.0
Predicted: 88155.21837856318, Actual: 115000.0
Predicted: 111177.17226127947, Actual: 159000.0
Predicted: 309399.4252983587, Actual: 315500.0
Predicted: 76938.28061563754, Actual: 75500.0
Predicted: 222791.40736765804, Actual: 311500.0
Predicted: 150296.49370439455, Actual: 146000.0
Predicted: 76041.92228463, Actual: 84500.0
Predicted: 117970.02944820898, Actual: 135500.0
Root Mean Squared Error: 32791.48388277687
